In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Для задачи Question Answering мы используем класс BertForQuestionAnswering из библиотеки transformers.

In [14]:
import pandas as pd
import json
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

Загрузим предъобученную модель BERT

In [15]:
model = BertForQuestionAnswering.from_pretrained('ai-forever/ruBert-base')

Some weights of the model checkpoint at ai-forever/ruBert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the mo

Загрузим и токенайзер

In [16]:
tokenizer = BertTokenizer.from_pretrained('ai-forever/ruBert-base')

# Загрузим датасет

In [22]:
# def get_data(path):
#   with open(path, 'rb') as f:
#     raw_data = json.load(f)

#   return raw_data


In [ ]:
# !wget -nc https://github.com/aspushkarev/nlp-test-task-2023/tree/main/data/raw/train.json

In [23]:
num_questions = 0
num_posible = 0
num_imposible = 0

In [41]:
#note: below code will only return questions wich have answers (i.e. not the ones flagged as imposible to answer)
def get_data(path):  
    #read each file and retrieve the contexts, qustions and answers
  with open(path, 'rb') as f:
    raw_data = json.load(f)

  contexts = []
  questions = []
  answers = []
  num_q = 0
  num_pos = 0
  num_imp = 0

  for group in raw_data:
    for paragraph in group:
      context = paragraph
      for qa in paragraph:
        question = qa
        num_q  = num_q  +1
        if qa:
            num_imp = num_imp +1
        else:
            num_pos = num_pos +1
        for answer in qa:
          contexts.append(context.lower())
          questions.append(question.lower())
          answers.append(answer)

  return num_q, num_pos, num_imp, contexts, questions, answers

In [42]:
num_q, num_pos, num_imp, train_contexts, train_questions, train_answers = get_data('train.json')
num_questions  = num_q
num_posible = num_pos
num_imposible  = num_imp

In [43]:
print(train_questions[0:10])
print(train_answers[0:10])

['i', 'd', 't', 'e', 'x', 't', 'l', 'a', 'b', 'e']
['i', 'd', 't', 'e', 'x', 't', 'l', 'a', 'b', 'e']


In [51]:

data = pd.read_json("train.json")
data.tail(2)

,id,text,label,extracted_part
1797,903507299,ДОКУМЕНТАЦИЯ О ЗАКУПКЕ ЗАПРОС ПРЕДЛОЖЕНИЙ В ЭЛ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."
1798,926039184,Государственная корпорацИя по атомной энергии ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."


In [76]:
# with open('train.json', 'rb') as f:
#     raw_data = json.load(f)

contexts = []
questions = []
answers = []
num_q = 0
num_pos = 0
num_imp = 0

for text in data['text']:
    num_q += 1
    contexts.append(text.lower())

for question in data['extracted_part']:
    num_pos += 1
    print(question)
    # questions.append(question.lower())
    # for paragraph in group:
    #   print(paragraph)
#       context = paragraph['context']
#       for qa in paragraph['qas']:
#         question = qa['question']
#         num_q  = num_q  +1
#         if qa['is_impossible'] == True:
#             num_imp = num_imp +1
#         else:
#             num_pos = num_pos +1
#         for answer in qa['answers']:
#           contexts.append(context.lower())
#           questions.append(question.lower())
#           answers.append(answer)

{'text': ['Размер обеспечения исполнения контракта 6593.25 Российский рубль'], 'answer_start': [1279], 'answer_end': [1343]}
{'text': ['Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта.'], 'answer_start': [1222], 'answer_end': [1318]}
{'text': ['Размер обеспечения исполнения контракта 10.00%'], 'answer_start': [1297], 'answer_end': [1343]}
{'text': ['Размер обеспечения исполнения контракта 10.00%'], 'answer_start': [1304], 'answer_end': [1350]}
{'text': ['Размер обеспечения исполнения контракта 10.00%'], 'answer_start': [1302], 'answer_end': [1348]}
{'text': ['Размер обеспечения исполнения контракта устанавливается в размере 5 (пять) процентов от цены, по которой заключается контракт и составляет ___'], 'answer_start': [1256], 'answer_end': [1398]}
{'text': ['Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта'], 'answer_start': [1298], 'answer_end': [1410]}
{'text': ['Размер обе

In [73]:
print(contexts[:3])
print(len(contexts))
print(num_q)

['извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 общая информация номер извещения 0328300032822000806 наименование объекта закупки поставка продуктов питания способ определения поставщика (подрядчика, исполнителя) открытый конкурс в бль порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 федерального закона № 44-фз. выбор способа обеспечения осуществляется участником закупки самостоятельно. срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 федерального закона № 44-фз. условия независимой гарантии в соответствии со ст. 45 федерального закона № 44-фз. реквизиты счета в соответствии с п.16 ч. 

In [ ]:
# question = "обеспечение исполнения контракта"
# answer_text = raw_data[:10]

In [ ]:
# print(answer_text)

In [ ]:
# # Apply the tokenizer to the input text, treating them as a text-pair.
# input_ids = tokenizer.encode(question, answer_text)

# print('The input has a total of {:} tokens.'.format(len(input_ids)))